
---

## Introduction

### Variable Stars

While most stars emit steady light, some show periodic variations in brightness—these are called [**variable stars**](http://en.wikipedia.org/wiki/Variable_star). Variability arises due to intrinsic pulsations or because the star is part of a [**binary system**](http://en.wikipedia.org/wiki/Binary_system) that causes eclipses. Studying such stars is vital for measuring cosmic distances. With upcoming surveys like [LSST](http://www.lsst.org/lsst), automating their analysis is increasingly important.

### The EROS1 Database

Our dataset comes from the [**EROS1 project**](http://eros.in2p3.fr/), which captured 400 photographic plates (1990–1994) of the [Large Magellanic Cloud](http://en.wikipedia.org/wiki/Large_Magellanic_Cloud) using the ESO-Schmidt 1m telescope. Each plate, covering a 5°×5° field, was taken in two bands (red: 630nm, blue: 385nm), digitized in Paris, and analyzed at IN2P3.

### Variable Star Selection

Out of 8 million stars, \~1% were flagged as unstable by an algorithm. These were manually labeled into types: **RR-Lyrae, Cepheid, Eclipsing Binary, Mira**, or **Other**. A total of **22,802** variables were found, reduced to **19,429** after data cleaning.

### Dataset Used

For modeling, “unclassified” types were excluded, and 30% of the remaining data (3,641 instances) was used for training. The dataset includes:

* [`train.csv`] static features
* [`train_varlength_features.csv.gz`] variable-length time series features (loaded via `get_train_data`)

---


# Exploratory data analysis

In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
pd.set_option('display.max_columns', None)

## Get access to the training data

In [9]:
import problem

X_df, y = problem.get_train_data()

ModuleNotFoundError: No module named 'problem'

## The static features

<span style="color:red">Red variables</span> are ids, labels, or other human-annotated features, so they should not be used as input in the classification. <span style="color:lightblue">Light blue variables</span> are legal but not likely to contribute information to the classification. Black variables are definitely discriminative.

<ul>
    <li> <code><b><span style="color:red">patch_id</span></b></code>: The catalogue is organized by tiles corresponding to 1 cm<sup>2</sup> patches on the plates, this is their id.
    <li> <code><b><span style="color:red">star_id_b, star_id_r</span></b></code>: The id of the star within the patch. <code>patch_id</code> and <code>star_id_b</code> or <code>star_id_r</code> identify the stars uniquely, and we use <code>[patch_id]_[star_id_b]</code> for indexing the stars in the pandas table.
    <li> <code><b>magnitude_b, magnitude_r</b></code>: The average apparent luminosity of the star (in two frequency bands). <a href="http://www.astro-tom.com/technical_data/magnitude_scale.htm">Magnitude</a> is a logarithmic measure, and the higher it is, the lower the apparent luminosity is.
    <li> <code><b><span style="color:lightblue">asc_d, asc_m, asc_s</span></b></code>: <a href="http://en.wikipedia.org/wiki/Celestial_coordinate_system">Celestial</a> <a href="http://en.wikipedia.org/wiki/Right_ascension">right ascension</a> (coordinate) of the star, measured in degrees, minutes, and seconds, respectively.
    <li> <code><b><span style="color:lightblue">dec_d, dec_m, dec_s</span></b></code>: <a href="http://en.wikipedia.org/wiki/Celestial_coordinate_system">Celestial</a> <a href="http://en.wikipedia.org/wiki/Declination">declination</a> (coordinate) of the star, measured in degrees, minutes, and seconds, respectively.
    <li> <code><b><span style="color:black">period</span></b></code>: the estimated period of the light curve. For the correct period, it should be divided by <code>div_period</code>
    <li> <code><b><span style="color:lightblue">frequency</span></b></code>: $1/$<code>period</code>, so it is redundant.
    <li> <code><b><span style="color:lightblue">num_points_good_b, num_points_good_r</span></b></code>: Number of good light curve measurements (some measurements can be corrupted). 
    <li> <code><b><span style="color:black">asym_b, asym_r</span></b></code>: Unknown semantics.
    <li> <code><b><span style="color:black">log_p_not_variable</span></b></code>: Logarithm of the estimated probability that the star is stable.
    <li> <code><b><span style="color:black">sigma_flux_b, sigma_flux_r</span></b></code>: The square root of the total variance of the light measurements (indicating the amplitude of the variability).
    <li> <code><b><span style="color:red">type</span></b></code>: The label to predict.
    <li> <code><b><span style="color:red ">quality</span></b></code>: Human-annotated measure of the quality of the time curve. The higher the better.
    <li> <code><b><span style="color:black ">div_period</span></b></code>: The algorithm that estimates the <code>period</code> sometimes finds a multiple of the period. These cases were human-detected, and the divisor was recorded. In principle this variable is not available automatically as an observable, but we are confident that it could be obtained automatically, so we allow it as an input.
</ul> 

In [3]:
X_df.head()

NameError: name 'X_df' is not defined

## The labels

In [ ]:
label_names = {1: 'binary', 2: 'cepheid', 3: 'rr_lyrae', 4: 'mira'}
labels = list(label_names.keys())
y_series = pd.Series(y).replace(label_names)
y_series.head()

In [ ]:
_ = y_series.value_counts().plot(kind="bar")

## Some classwise histograms and scatterplots

In [ ]:
colors = ['r', 'b', 'g', 'm']
def plot_classwise_normalized(feature, bins=None):
    if bins is None:
        bins = np.linspace(X_df[feature].min(), X_df[feature].max(), 15)
    for label, color in zip(labels, colors):
        plt.hist(X_df[y == label][feature].values, density=True, bins=bins, 
                 alpha=0.5, color=color)

In [ ]:
plot_classwise_normalized('period')

In [ ]:
plot_classwise_normalized('period', bins=np.linspace(0, 50, 15))

Observe the aliasing below.

In [ ]:
plot_classwise_normalized('period', bins=np.linspace(0, 5, 15))

In [ ]:
X_df['real_period'] = X_df['period'] / X_df['div_period']

In [ ]:
plot_classwise_normalized('real_period')

In [ ]:
plot_classwise_normalized('real_period', bins=np.linspace(0, 50, 15))

In [ ]:
plot_classwise_normalized('real_period', bins=np.linspace(0, 5, 15))

In [ ]:
plot_classwise_normalized('magnitude_b')

In [ ]:
plot_classwise_normalized('magnitude_r')

In [ ]:
plot_classwise_normalized('asym_b')

In [ ]:
plot_classwise_normalized('asym_r')

In [ ]:
plot_classwise_normalized('sigma_flux_b')

In [ ]:
plot_classwise_normalized('sigma_flux_b', bins=np.linspace(0, 1000, 15))

In [ ]:
plot_classwise_normalized('sigma_flux_b', bins=np.linspace(0, 100, 15))

In [ ]:
plot_classwise_normalized('sigma_flux_r')

In [ ]:
plot_classwise_normalized('sigma_flux_r', bins=np.linspace(0, 1000, 15))

In [ ]:
plot_classwise_normalized('sigma_flux_r', bins=np.linspace(0, 100, 15))

In [ ]:
plot_classwise_normalized('log_p_not_variable')

In [ ]:
colors = ['r', 'b', 'g', 'm']
def plot_classwise_scatter(feature1, feature2, range1=None, range2=None):
    if range1 is None:
        range1 = [X_df[feature1].min(), X_df[feature1].max()]
    if range2 is None:
        range2 = [X_df[feature2].min(), X_df[feature2].max()]
    for label, color in zip(labels, colors):
        plt.xlim(range1[0], range1[1])
        plt.ylim(range2[0], range2[1])
        plt.scatter(X_df[y == label][feature1], 
                    X_df[y == label][feature2],
            alpha=0.3, s=80, c=color, marker='.');

In [ ]:
plot_classwise_scatter('magnitude_b', 'magnitude_r')

In [ ]:
plot_classwise_scatter('magnitude_b', 'real_period', range1=None, range2=[0,10])

## The time series

Each column contains a list of floating point numbers.

<ul>
    <li> <code><b><span style="color:black">time_points_b, time_points_r</span></b></code>: The time (in unit of days) when the photos were taken. Note that the filters had to be changed so the time points of the blue and red frequency band are slighty different.
    <li> <code><b><span style="color:black">light_points_b, light_points_r</span></b></code>: The light points measured at the time points.
    <li> <code><b><span style="color:black">error_points_b, error_points_r</span></b></code>: Uncertainties (error bars) on the light measurements.
    <li> <code><b><span style="color:lightblue">bkg_points_, bkg_points_r</span></b></code>: Background noise measured at the time points.
    <li> <code><b><span style="color:lightblue">polltn_points_b, polltn_points_r</span></b></code>: Pollution noise measured at the time points.
</ul> 

In [ ]:
X_df.hist('num_points_good_r')
print(min(X_df['num_points_good_b']))
print(max(X_df['num_points_good_b']))
print(min(X_df['num_points_good_r']))
print(max(X_df['num_points_good_r']))

### Plotting time curves

Set the patch id and star id below.

In [ ]:
patch_id = 98
star_id_b = 477

def star_key(slab_id, star_id_b):
    return str(slab_id) + '_' + str(star_id_b)

In [ ]:
X_df = X_df.set_index(X_df.apply(lambda row: "%d_%d" % (row['patch_id'], row['star_id_b']), axis=1))

In [ ]:
time_points = X_df.loc[star_key(patch_id, star_id_b)]['time_points_b']
light_points = X_df.loc[star_key(patch_id, star_id_b)]['light_points_b']
error_points = X_df.loc[star_key(patch_id, star_id_b)]['error_points_b']
plt.errorbar(time_points, light_points, yerr=error_points, fmt='o');

The raw measurements seem rather messy. The scatter of the plots is visibly larger than the measurement uncertainty (which makes it, by definition, a variable star), but there is no visible periodicity. We can use the estimated period to overplot several periods of the curve ("fold" the time series) using the following function.

In [ ]:
def fold_time_series(time_point, period, div_period):
    real_period = period / div_period
    return time_point % real_period  # modulo real_period

In [ ]:
period = X_df.loc[star_key(patch_id, star_id_b)]['period']
div_period = X_df.loc[star_key(patch_id, star_id_b)]['div_period']
print(period, div_period)

In [ ]:
time_points_folded = [fold_time_series(time_point, period, div_period) 
                      for time_point in time_points]

The resulting curve has a characteristic signature. 

In [ ]:
plt.gca().invert_yaxis()
plt.errorbar(time_points_folded, light_points, yerr=error_points, fmt='o');

The feature extractor

The input data are stored in a dataframe. To go from a dataframe to a numpy array we will
a scikit-learn column transformer. The first example we will write will just consist in
selecting a subset of columns we want to work with.

In [ ]:
cols = [
    'magnitude_b', 
    'magnitude_r',
    'period',
    'asym_b', 
    'asym_r', 
    'log_p_not_variable', 
    'sigma_flux_b', 
    'sigma_flux_r', 
    'quality', 
    'div_period',
]

from sklearn.compose import make_column_transformer

transformer = make_column_transformer(
    ('passthrough', cols)
)

X_array = transformer.fit_transform(X_df)

In [ ]:
X_array

Let's look at how to transform line curves.

The following feature extractor takes the light curve, bins it into <code>num_bins</code> bins, and return the bin means. It works with one band at a time.

In [ ]:
def fold_time_series(time_point, period, div_period):
    return (time_point -
            (time_point // (period / div_period)) * period / div_period)


def get_bin_means(X_df, num_bins, band):
    feature_array = np.empty((len(X_df), num_bins))

    for k, (_, x) in enumerate(X_df.iterrows()):
        period = x['period']
        div_period = x['div_period']
        real_period = period / div_period
        bins = [i * real_period / num_bins for i in range(num_bins + 1)]

        time_points = np.array(x['time_points_' + band])
        light_points = np.array(x['light_points_' + band])
        time_points_folded = \
            np.array([fold_time_series(time_point, period, div_period)
                      for time_point in time_points])
        time_points_folded_digitized = \
            np.digitize(time_points_folded, bins) - 1

        for i in range(num_bins):
            this_light_points = light_points[time_points_folded_digitized == i]
            if len(this_light_points) > 0:
                feature_array[k, i] = np.mean(this_light_points)
            else:
                feature_array[k, i] = np.nan  # missing

    return feature_array


get_bin_means(X_df.iloc[:2], 5, 'r')

For this we will use a funtion transformer that will get applied to both red and blues curves.

In [ ]:
from sklearn.preprocessing import FunctionTransformer

transformer_r = FunctionTransformer(
    lambda X_df: get_bin_means(X_df, 5, 'r')
)

transformer_b = FunctionTransformer(
    lambda X_df: get_bin_means(X_df, 5, 'b')
)

transformer = make_column_transformer(
    (transformer_r, ['period', 'div_period', 'time_points_r', 'light_points_r']),
    (transformer_b, ['period', 'div_period', 'time_points_b', 'light_points_b']),
)

X_array = transformer.fit_transform(X_df)
X_array.shape

Combined with some static features and plugged into a random forest it reads;

In [ ]:
import numpy as np

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer


def fold_time_series(time_point, period, div_period):
    return (time_point -
            (time_point // (period / div_period)) * period / div_period)


def get_bin_means(X_df, num_bins, band):
    feature_array = np.empty((len(X_df), num_bins))

    for k, (_, x) in enumerate(X_df.iterrows()):
        period = x['period']
        div_period = x['div_period']
        real_period = period / div_period
        bins = [i * real_period / num_bins for i in range(num_bins + 1)]

        time_points = np.array(x['time_points_' + band])
        light_points = np.array(x['light_points_' + band])
        time_points_folded = \
            np.array([fold_time_series(time_point, period, div_period)
                      for time_point in time_points])
        time_points_folded_digitized = \
            np.digitize(time_points_folded, bins) - 1

        for i in range(num_bins):
            this_light_points = light_points[time_points_folded_digitized == i]
            if len(this_light_points) > 0:
                feature_array[k, i] = np.mean(this_light_points)
            else:
                feature_array[k, i] = np.nan  # missing

    return feature_array


transformer_r = FunctionTransformer(
    lambda X_df: get_bin_means(X_df, 5, 'r')
)

transformer_b = FunctionTransformer(
    lambda X_df: get_bin_means(X_df, 5, 'b')
)

cols = [
    'magnitude_b',
    'magnitude_r',
    'period',
    'asym_b',
    'asym_r',
    'log_p_not_variable',
    'sigma_flux_b',
    'sigma_flux_r',
    'quality',
    'div_period',
]

common = ['period', 'div_period']
transformer = make_column_transformer(
    (transformer_r, common + ['time_points_r', 'light_points_r']),
    (transformer_b, common + ['time_points_b', 'light_points_b']),
    ('passthrough', cols)
)

pipe = make_pipeline(
    transformer,
    SimpleImputer(strategy='most_frequent'),
    RandomForestClassifier(max_depth=5, n_estimators=10)
)


def get_estimator():
    return pipe

## Testing using a scikit-learn pipeline

In [ ]:
import problem
from sklearn.model_selection import cross_val_score

X_df, y = problem.get_train_data()

scores = cross_val_score(get_estimator(), X_df, y, cv=2, scoring='accuracy')
print(scores)